# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install .`

In [1]:
# %pip install ..

In [2]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pd.read_csv(demonstration_fp)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1933-03-04,12:41:26,2022-08-02 16:42:16
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1923-05-05,14:08:00,2022-08-10 13:32:45
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1925-12-06,10:51:26,2022-08-14 19:51:11
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1923-09-07,17:05:01,2022-07-30 08:22:05
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1919-12-07,14:18:06,2022-07-24 07:45:26


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [4]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv(demonstration_fp, dtype=dtypes)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1933-03-04,12:41:26,2022-08-02 16:42:16
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1923-05-05,14:08:00,2022-08-10 13:32:45
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1925-12-06,10:51:26,2022-08-14 19:51:11
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1923-09-07,17:05:01,2022-07-30 08:22:05
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1919-12-07,14:18:06,2022-07-24 07:45:26


### Dates, times, and datetimes

One exception to the above is dates, times, and datetimes. Here, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

In [5]:
df["Birthday"] = [dt.date.fromisoformat(x) for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) for x in df["Board time"]]
df["Married since"] = [dt.datetime.fromisoformat(x) for x in df["Married since"]]

Now, let's check the data types of our DataFrame:

In [6]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [7]:
df["Birthday"][0]

datetime.date(1933, 3, 4)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [8]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:126: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [9]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameter

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [10]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [11]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,66,"%{ .gw`rzB& {2%a,/ dP",male,25,0,17.810179,<NA>,S,1904-11-07,12:16:11,2022-07-25 16:18:29
1,891,8G6@9 1;Ub$@?+5X5K4]36=u,female,60,0,21.465246,<NA>,C,1906-01-08,13:22:08,2022-07-21 18:11:19
2,81,QS;?e}d.FRZ[( N+0ODbaP <PPjjQ.0}z>mkJ,female,1,1,4.889082,<NA>,S,1910-03-01,11:57:08,2022-07-30 05:06:17
3,79,"oyEg8%YU_V,Mc=$JLWQb8^-W^Y#'H4M87T6>=k2Hh'K22...",male,18,0,5.136463,<NA>,S,1928-11-26,18:27:22,2022-07-19 22:02:08
4,220,"nR Jtr-t.&c5vjjs*j,k3!B""~$ Wr'H}8$ $yu&",male,69,0,27.742367,GvvWr,S,1914-03-31,12:25:20,2022-07-27 03:52:00


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [13]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"$6 iqVgvvk]'eVc~1*/ofRHlGk3yx/,1fV F43'#Dxe P...",male,<NA>,0,131.398550,<NA>,Q,1921-04-18,17:14:41,2022-08-10 07:07:18
1,2,qFbiSf^ C\7V8]|km1#e'|lK*$'-Ti xrR^4$:KlHRmOA}...,male,9,0,18.321457,<NA>,S,1933-08-04,13:16:25,2022-07-26 00:11:42
2,3,"%fzcAF/8. 9 85]4X&m,b;8,q{B5*v>U`",male,77,0,44.986649,<NA>,S,1937-11-09,13:01:37,2022-08-11 11:36:50
3,4,TtoL*#H_OWqQi'uyEdi &rz8{h7BMJQV)+lW^PWnSxiM (...,male,36,2,15.271091,<NA>,C,1916-06-15,18:29:38,2022-08-02 21:42:58
4,5,OU)~KRs;[S[ sh|c:ub=j=m4ilzcU nqviVts~=kiDnWCn...,male,48,0,81.918371,<NA>,C,1931-08-26,18:14:16,2022-07-15 20:37:14


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [14]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Juan Peters,male,24,0,8.732195,<NA>,C,1925-10-08,10:43:00,2022-08-14 04:40:52
1,2,Christine Neal,male,28,1,22.160158,<NA>,S,1906-03-07,11:29:18,2022-08-09 04:26:25
2,3,Natalie Harris,female,72,0,25.705391,<NA>,S,1916-09-19,12:48:10,2022-07-19 15:39:13
3,4,David Hart,male,<NA>,0,114.412392,<NA>,S,1932-10-14,12:23:23,2022-08-11 16:41:38
4,5,Lindsay Floyd,female,<NA>,1,28.263488,"C>94F7""",S,1925-01-16,13:17:03,2022-08-08 05:20:30


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [15]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Justin Smith,female,21,0,1.027887,<NA>,C,1922-08-04,11:17:10,2022-08-11 18:41:15
1,2,Donna Hendrix,female,20,0,0.336495,<NA>,S,1905-03-07,16:08:16,2022-07-30 13:00:00
2,3,Angela Morgan,male,35,1,0.414547,<NA>,S,1937-09-07,11:51:45,2022-08-13 03:01:19
3,4,Walter Caldwell,female,34,0,1.282756,<NA>,S,1929-07-21,15:08:34,2022-08-13 23:01:11
4,5,Brianna Johnson,male,35,2,2.003559,T8?\ xT<,C,1923-07-31,13:13:55,2022-08-02 15:01:03


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [16]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution([(r"[ABCDEF]", 1), (r"\d{2,3}", 0.9)])  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Christina Rodriguez,male,<NA>,0,3.734902,<NA>,S,1908-10-27,17:58:08,2022-08-12 12:43:43
1,2,Patrick Taylor,female,28,4,1.093146,<NA>,Q,1939-01-02,11:33:08,2022-08-01 03:40:50
2,3,Raymond Dean,female,<NA>,1,5.107747,<NA>,S,1916-07-30,16:34:15,2022-07-17 00:44:06
3,4,Michelle Page,male,<NA>,0,1.889658,<NA>,S,1916-04-08,13:42:25,2022-08-14 06:57:15
4,5,Jessica Graham,male,<NA>,0,3.862991,<NA>,Q,1937-01-01,11:12:17,2022-08-03 01:29:43
5,6,Matthew Wright,female,<NA>,1,0.328751,<NA>,S,1927-10-23,17:38:51,2022-07-20 17:05:49
6,7,Philip Smith,male,22,2,1.374917,B766,S,1914-03-09,15:41:24,2022-08-11 23:09:44
7,8,Jason Ferguson,female,<NA>,0,3.757007,F199,Q,1928-05-08,16:21:29,2022-08-02 03:06:56
8,9,Kendra Perkins,male,26,0,0.372023,<NA>,C,1921-01-26,14:43:55,2022-07-17 13:24:53
9,10,Brian Ramirez,female,<NA>,0,0.868110,<NA>,S,1908-03-04,18:32:46,2022-08-02 16:14:29
